## Creating folders for each WARENGRUPPE along with the product data for that WR

This is a one-time used code to structure the data so that products for each product group are in their separate folder along with their prompt format and allowed keys as well as their output csv and json files. This will structure the data so that each product group is treated individually based on its attributes. 

In [1]:
## Importing libraries and initializing values class to import the relevant variables
import pandas as pd
from values import *
import os
# from dbfread import DBF

val = Values()
encoding_ = 'cp850'
encoding_errors='replace'

In [ ]:
marketing_art = pd.read_excel('/Users/maralsheikhzadeh/Documents/Codes/P - Artikel Attribute/tischdecken.xlsx',engine='openpyxl')
mar = pd.read_csv(val.marketing_artikel,sep=';',encoding=encoding_,on_bad_lines='skip')

marketing_art = pd.merge(marketing_art,mar,on='NUMMER',how='left')
marketing_art = marketing_art.drop_duplicates(subset='NUMMER')

In [ ]:
marketing_art

In [4]:
## loading data from warengruppe list, the products in the webshop and the marketing artikel list that connects the other two datas together
warengr = pd.read_csv(val.wr_filepath,sep=';',encoding_errors=encoding_errors,encoding=encoding_)
shop = pd.read_excel(val.shop_file_path,engine='openpyxl')
shop2 = pd.read_excel('/Users/maralsheikhzadeh/Documents/Codes/Data/All Exports/Artikel/export_report-01-aa.xlsx',engine='openpyxl')
shop3 = pd.read_excel('/Users/maralsheikhzadeh/Documents/Codes/Data/All Exports/Artikel/export-01-aa-JG.xlsx',engine='openpyxl')



## We need the first part of the StoreId number to connect with the marketing artikel list
# shop['NUMMER'] = shop['StoreId'].str.split().str[0]

# cat_data = pd.read_csv(val.cat_data,delimiter=';',encoding_errors=encoding_errors)

shop = pd.concat([shop,shop2,shop3])
shop = shop.drop_duplicates(subset='StoreId')

In [5]:
shop['NUMMER'] = shop['StoreId'].str.strip()
shop['NUMMER'] = shop['NUMMER'].str.split().str[0]
shop['Var_Text'] = shop['Var_Text'].fillna("")

### Data Processing

In [6]:

# marketing_art['WARENGR'] = marketing_art['WARENGR'].astype(str)
# marketing_art['WARENGR'] = marketing_art['WARENGR'].str.replace('.0','')
# marketing_art['WARENGR'] = marketing_art['WARENGR'].str.strip()


# marketing_art = marketing_art[['WM','NUMMER','GROESSE','FARBE','WG_CODE']]
marketing_art = marketing_art[(marketing_art['WM'].isna())|(marketing_art['WM']=="")]
marketing_art['NUMBER'] = marketing_art['NUMMER'].str.ljust(8) + marketing_art['GROESSE'].str.ljust(4) + marketing_art['FARBE'].str.ljust(2)
marketing_art['NUMBER'] = marketing_art['NUMBER'].str.strip()
# mark_bez_df['NUMMER'] = mark_bez_df['NUMMER'].str.strip()

# warengr['WAREN_GRP'] = warengr['WAREN_GRP'].astype(str)
# warengr['WAREN_GRP'] = warengr['WAREN_GRP'].str.replace('.0','')
# warengr['WAREN_GRP'] = warengr['WAREN_GRP'].str.strip()



In [7]:
marketing_art = marketing_art[['NUMMER','NUMBER',
 'BANAME',
 'WGNAME',
 'WG_CODE',
 'WG_NAME',
 'WM',
 'GROESSE',
 'FARBE'
 ]]

In [ ]:
artikels = pd.merge(marketing_art,shop,how='left',on="NUMMER")
artikels = artikels[['WM','NUMMER','GROESSE','FARBE', 'NUMBER','WG_CODE','Name','Beschreibung','Var_Text']]
artikels = artikels.rename(columns={'Name':'NAME','Beschreibung':'NET_BESCHREIBUNG'})
artikels['NET_BESCHREIBUNG'] = artikels['NET_BESCHREIBUNG'].str.replace(' ',' ',regex=True)
artikels['NET_BESCHREIBUNG'] = artikels['NET_BESCHREIBUNG'].str.replace(r'(<[^>]*>)',' ',regex=True,case=False)
artikels['NET_BESCHREIBUNG'] = artikels['NET_BESCHREIBUNG'].str.replace(r'&nbsp_|&nbsp;|&Oslash',' ',regex=True)
artikels['BESCHREIBUNG'] = artikels['NAME'] + ' - ' + artikels['Var_Text'] + ' - ' + artikels['NET_BESCHREIBUNG']
artikels = artikels.drop_duplicates('NUMMER')
artikels = artikels.dropna(subset=['NET_BESCHREIBUNG'])
artikels = artikels[['NUMMER','WG_CODE','NAME','BESCHREIBUNG']]
artikels = artikels.rename(columns={"BESCHREIBUNG":'NET_BESCHREIBUNG'})
artikels.to_csv("tischdecke-online.csv",sep=';',encoding='utf-8',index=False)
artikels[artikels['NUMMER']=='047HX11']